In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [7]:
ant_master = pickle.load( open( "xalan_master.p", "rb" ) )

In [8]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xalan,2.6.0,org.apache.xalan.xsltc.compiler.FilterExpr,7,3,0,19,35,0,0,...,1.00000,1,0.922078,0.306122,2,9,31.000000,3,1.5714,1
1,xalan,2.6.0,org.apache.xpath.operations.Operation,12,2,14,22,23,0,17,...,1.00000,2,0.731707,0.218750,1,7,11.250000,5,1.5833,1
2,xalan,2.6.0,org.apache.xalan.xsltc.runtime.BasisLibrary,69,1,0,36,198,2330,20,...,0.30303,0,0.000000,0.102076,0,0,36.217391,49,4.2464,3
3,xalan,2.6.0,org.apache.xpath.objects.XString,49,3,2,42,100,1176,33,...,0.00000,1,0.606838,0.122222,3,21,14.081633,16,1.8571,3
4,xalan,2.6.0,org.apache.xalan.xsltc.StripFilter,1,1,0,11,1,0,11,...,0.00000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,xalan,2.4.0,org.apache.xpath.patterns.NodeTestFilter,1,1,0,1,1,0,0,...,0.00000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
719,xalan,2.4.0,org.apache.xalan.xsltc.compiler.LangCall,4,4,0,19,17,0,0,...,1.00000,2,0.969697,0.375000,3,8,19.500000,2,1.0000,0
720,xalan,2.4.0,org.apache.xalan.xsltc.dom.FilterIterator,9,2,0,5,24,0,1,...,1.00000,2,0.555556,0.288889,1,5,10.444444,3,1.2222,0
721,xalan,2.4.0,org.apache.xalan.xsltc.compiler.RelativePathPa...,1,5,3,7,2,0,6,...,0.00000,0,1.000000,1.000000,0,0,3.000000,0,0.0000,0


In [10]:
test = ant_master[ant_master['version']=='2.6.0']
test

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xalan,2.6.0,org.apache.xalan.xsltc.compiler.FilterExpr,7,3,0,19,35,0,0,...,1.000000,1,0.922078,0.306122,2,9,31.000000,3,1.5714,1
1,xalan,2.6.0,org.apache.xpath.operations.Operation,12,2,14,22,23,0,17,...,1.000000,2,0.731707,0.218750,1,7,11.250000,5,1.5833,1
2,xalan,2.6.0,org.apache.xalan.xsltc.runtime.BasisLibrary,69,1,0,36,198,2330,20,...,0.303030,0,0.000000,0.102076,0,0,36.217391,49,4.2464,3
3,xalan,2.6.0,org.apache.xpath.objects.XString,49,3,2,42,100,1176,33,...,0.000000,1,0.606838,0.122222,3,21,14.081633,16,1.8571,3
4,xalan,2.6.0,org.apache.xalan.xsltc.StripFilter,1,1,0,11,1,0,11,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,xalan,2.6.0,org.apache.xalan.xsltc.compiler.VariableRef,2,4,0,17,25,1,1,...,0.000000,0,0.987013,0.625000,2,3,67.500000,9,4.5000,0
881,xalan,2.6.0,org.apache.xml.serializer.AttributesImplSerial...,6,2,0,6,20,0,6,...,0.666667,0,0.827586,0.416667,1,2,19.166667,3,2.0000,1
882,xalan,2.6.0,org.apache.xpath.objects.XNodeSetForDOM,7,5,0,5,18,0,1,...,0.000000,0,0.978417,0.285714,3,14,15.285714,1,0.4286,0
883,xalan,2.6.0,org.apache.xpath.compiler.PsuedoNames,1,1,0,0,2,0,0,...,0.000000,0,0.000000,1.000000,0,0,3.000000,0,0.0000,0


In [12]:
train = ant_master[ant_master['version']!='2.6.0']
train

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xalan,2.5.0,org.apache.xpath.res.XPATHErrorResources_sv,3,4,0,1,4,1,0,...,0.000000,0,0.976190,1.000000,0,0,618.333333,1,0.3333,1
1,xalan,2.5.0,org.apache.xalan.xsltc.compiler.TransletOutput,5,3,0,20,33,0,0,...,1.000000,1,0.933333,0.333333,2,7,33.800000,6,1.8000,0
2,xalan,2.5.0,org.apache.xalan.lib.Extensions,13,1,0,11,54,76,1,...,0.500000,1,0.000000,0.208333,0,0,27.230769,5,1.9231,1
3,xalan,2.5.0,org.apache.xml.serializer.ToHTMLStream,28,3,0,12,104,222,1,...,0.888889,5,0.866667,0.196296,2,26,87.250000,3,1.1429,4
4,xalan,2.5.0,org.apache.xpath.functions.FuncConcat,4,7,0,7,13,6,0,...,0.000000,0,0.960000,0.500000,3,8,19.250000,1,0.7500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,xalan,2.4.0,org.apache.xpath.patterns.NodeTestFilter,1,1,0,1,1,0,0,...,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
719,xalan,2.4.0,org.apache.xalan.xsltc.compiler.LangCall,4,4,0,19,17,0,0,...,1.000000,2,0.969697,0.375000,3,8,19.500000,2,1.0000,0
720,xalan,2.4.0,org.apache.xalan.xsltc.dom.FilterIterator,9,2,0,5,24,0,1,...,1.000000,2,0.555556,0.288889,1,5,10.444444,3,1.2222,0
721,xalan,2.4.0,org.apache.xalan.xsltc.compiler.RelativePathPa...,1,5,3,7,2,0,6,...,0.000000,0,1.000000,1.000000,0,0,3.000000,0,0.0000,0


In [13]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [14]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,3,4,0,1,4,1,0,1,2,1.428571,1872,0.000000,0,0.976190,1.000000,0,0,618.333333,1,0.3333
1,5,3,0,20,33,0,0,20,4,0.500000,176,1.000000,1,0.933333,0.333333,2,7,33.800000,6,1.8000
2,13,1,0,11,54,76,1,10,9,0.875000,371,0.500000,1,0.000000,0.208333,0,0,27.230769,5,1.9231
3,28,3,0,12,104,222,1,11,17,0.855967,2480,0.888889,5,0.866667,0.196296,2,26,87.250000,3,1.1429
4,4,7,0,7,13,6,0,7,3,2.000000,81,0.000000,0,0.960000,0.500000,3,8,19.250000,1,0.7500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,1,1,0,1,1,0,0,1,1,2.000000,1,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
719,4,4,0,19,17,0,0,19,4,0.666667,84,1.000000,2,0.969697,0.375000,3,8,19.500000,2,1.0000
720,9,2,0,5,24,0,1,4,9,0.625000,106,1.000000,2,0.555556,0.288889,1,5,10.444444,3,1.2222
721,1,5,3,7,2,0,6,1,0,2.000000,4,0.000000,0,1.000000,1.000000,0,0,3.000000,0,0.0000


In [15]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,7,3,0,19,35,0,0,19,6,0.083333,226,1.000000,1,0.922078,0.306122,2,9,31.000000,3,1.5714
1,12,2,14,22,23,0,17,7,12,0.363636,149,1.000000,2,0.731707,0.218750,1,7,11.250000,5,1.5833
2,69,1,0,36,198,2330,20,18,64,1.005793,2601,0.303030,0,0.000000,0.102076,0,0,36.217391,49,4.2464
3,49,3,2,42,100,1176,33,12,46,1.000000,740,0.000000,1,0.606838,0.122222,3,21,14.081633,16,1.8571
4,1,1,0,11,1,0,11,1,1,2.000000,1,0.000000,0,0.000000,1.000000,0,0,0.000000,1,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,2,4,0,17,25,1,1,16,2,2.000000,137,0.000000,0,0.987013,0.625000,2,3,67.500000,9,4.5000
881,6,2,0,6,20,0,6,0,5,0.800000,124,0.666667,0,0.827586,0.416667,1,2,19.166667,3,2.0000
882,7,5,0,5,18,0,1,4,7,0.000000,115,0.000000,0,0.978417,0.285714,3,14,15.285714,1,0.4286
883,1,1,0,0,2,0,0,0,1,2.000000,10,0.000000,0,0.000000,1.000000,0,0,3.000000,0,0.0000


In [16]:
y_train = train['bug']
y_test = test['bug']

In [17]:
y_train

0      1
1      0
2      1
3      4
4      1
      ..
718    0
719    0
720    0
721    0
722    1
Name: bug, Length: 1526, dtype: int64

In [18]:
y_test

0      1
1      1
2      3
3      3
4      0
      ..
880    0
881    1
882    0
883    0
884    1
Name: bug, Length: 885, dtype: int64

In [19]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [20]:
x_train_np

array([[3.00000000e+00, 4.00000000e+00, 0.00000000e+00, ...,
        6.18333333e+02, 1.00000000e+00, 3.33300000e-01],
       [5.00000000e+00, 3.00000000e+00, 0.00000000e+00, ...,
        3.38000000e+01, 6.00000000e+00, 1.80000000e+00],
       [1.30000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
        2.72307692e+01, 5.00000000e+00, 1.92310000e+00],
       ...,
       [9.00000000e+00, 2.00000000e+00, 0.00000000e+00, ...,
        1.04444444e+01, 3.00000000e+00, 1.22220000e+00],
       [1.00000000e+00, 5.00000000e+00, 3.00000000e+00, ...,
        3.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.70000000e+01, 4.00000000e+00, 2.00000000e+00, ...,
        1.54074074e+01, 5.00000000e+00, 1.59260000e+00]])

In [21]:
x_test_np

array([[ 7.        ,  3.        ,  0.        , ..., 31.        ,
         3.        ,  1.5714    ],
       [12.        ,  2.        , 14.        , ..., 11.25      ,
         5.        ,  1.5833    ],
       [69.        ,  1.        ,  0.        , ..., 36.2173913 ,
        49.        ,  4.2464    ],
       ...,
       [ 7.        ,  5.        ,  0.        , ..., 15.28571429,
         1.        ,  0.4286    ],
       [ 1.        ,  1.        ,  0.        , ...,  3.        ,
         0.        ,  0.        ],
       [36.        ,  1.        ,  0.        , ..., 24.02777778,
         7.        ,  1.1944    ]])

In [22]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [23]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [24]:
train_x

array([[[3.00000000e+00, 4.00000000e+00, 0.00000000e+00, ...,
         6.18333333e+02, 1.00000000e+00, 3.33300000e-01]],

       [[5.00000000e+00, 3.00000000e+00, 0.00000000e+00, ...,
         3.38000000e+01, 6.00000000e+00, 1.80000000e+00]],

       [[1.30000000e+01, 1.00000000e+00, 0.00000000e+00, ...,
         2.72307692e+01, 5.00000000e+00, 1.92310000e+00]],

       ...,

       [[9.00000000e+00, 2.00000000e+00, 0.00000000e+00, ...,
         1.04444444e+01, 3.00000000e+00, 1.22220000e+00]],

       [[1.00000000e+00, 5.00000000e+00, 3.00000000e+00, ...,
         3.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[2.70000000e+01, 4.00000000e+00, 2.00000000e+00, ...,
         1.54074074e+01, 5.00000000e+00, 1.59260000e+00]]])

In [25]:
test_y

array([1, 1, 3, 3, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 3, 0, 0, 0, 2, 2, 6, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       3, 0, 1, 3, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 2, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 2, 0, 2,
       0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 1, 1, 1, 2, 1, 1, 0, 2, 0, 1, 0,
       2, 1, 0, 1, 0, 2, 2, 0, 1, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 0, 0, 1, 0, 0, 2, 1, 1, 5,
       1, 2, 1, 2, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 3, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 2, 0, 1, 2, 0,
       1, 1, 0, 4, 0, 1, 0, 4, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 6, 1, 0, 2, 0, 1, 3, 1, 2, 0, 2, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 1, 0, 0, 1,

In [26]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [27]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])

Epoch 1/100
16/16 [==============================] - 8s 27ms/step - loss: 0.8432 - mse: 0.8432 - mae: 0.4644 - root_mean_squared_error: 0.9183 - mean_squared_logarithmic_error: 0.2213
Epoch 2/100
16/16 [==============================] - 0s 8ms/step - loss: 0.7188 - mse: 0.7188 - mae: 0.5324 - root_mean_squared_error: 0.8478 - mean_squared_logarithmic_error: 0.1762
Epoch 3/100
16/16 [==============================] - 0s 8ms/step - loss: 0.6622 - mse: 0.6622 - mae: 0.5878 - root_mean_squared_error: 0.8137 - mean_squared_logarithmic_error: 0.1792
Epoch 4/100
16/16 [==============================] - 0s 8ms/step - loss: 0.6342 - mse: 0.6342 - mae: 0.5433 - root_mean_squared_error: 0.7963 - mean_squared_logarithmic_error: 0.1592
Epoch 5/100
16/16 [==============================] - 0s 8ms/step - loss: 0.6080 - mse: 0.6080 - mae: 0.5413 - root_mean_squared_error: 0.7797 - mean_squared_logarithmic_error: 0.1619
Epoch 6/100
16/16 [==============================] - 0s 8ms/step - loss: 0.5892 - ms

Epoch 44/100
16/16 [==============================] - 0s 11ms/step - loss: 0.4881 - mse: 0.4881 - mae: 0.4554 - root_mean_squared_error: 0.6986 - mean_squared_logarithmic_error: 0.1328
Epoch 45/100
16/16 [==============================] - 0s 16ms/step - loss: 0.4811 - mse: 0.4811 - mae: 0.4628 - root_mean_squared_error: 0.6936 - mean_squared_logarithmic_error: 0.1318
Epoch 46/100
16/16 [==============================] - 0s 10ms/step - loss: 0.4784 - mse: 0.4784 - mae: 0.4518 - root_mean_squared_error: 0.6917 - mean_squared_logarithmic_error: 0.1308
Epoch 47/100
16/16 [==============================] - 0s 10ms/step - loss: 0.4685 - mse: 0.4685 - mae: 0.4530 - root_mean_squared_error: 0.6844 - mean_squared_logarithmic_error: 0.1259
Epoch 48/100
16/16 [==============================] - 0s 13ms/step - loss: 0.4567 - mse: 0.4567 - mae: 0.4503 - root_mean_squared_error: 0.6758 - mean_squared_logarithmic_error: 0.1244
Epoch 49/100
16/16 [==============================] - 0s 13ms/step - loss: 

In [28]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6007 (pid 27262), started 2:03:38 ago. (Use '!kill 27262' to kill it.)

In [29]:
score = model.evaluate(test_x, test_y)
score

28/28 [==============================] - 2s 5ms/step - loss: 0.7898 - mse: 0.7898 - mae: 0.5790 - root_mean_squared_error: 0.8887 - mean_squared_logarithmic_error: 0.1762


[0.7897658944129944,
 0.7897658944129944,
 0.5789976119995117,
 0.8886877298355103,
 0.17618031799793243]